This notebook provides code to test all models with validation datasets (either held out test sets or external validation datasets).

In [1]:
# import statements 
import sys
sys.path.insert(1, './main_classes/')

from wrapper import run_bioautomated
from integrated_design_helpers import *
from generic_automl_classes import convert_generic_input, read_in_data_file
from generic_deepswarm import print_summary
from transfer_learning_helpers import transform_classification_target, transform_regression_target, fit_final_deepswarm_model
from generic_tpot import reformat_data_traintest

import scipy.stats as sp
from keras.initializers import glorot_uniform
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import autokeras
import torch
import pickle


Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


# Example 1: Transfer Learning on a DeepSwarm Model 

In [5]:
# Load DeepSwarm Model and freeze all except last two layers (randomly chose this - feel free to customize)
final_model_path = './exemplars/rbs/outputs/deepswarm/binary_classification/'
final_model_name = 'deepswarm_deploy_model.h5'
# get sequences with help from https://stackoverflow.com/questions/53183865/unknown-initializer-glorotuniform-when-loading-keras-model
with CustomObjectScope({'GlorotUniform': glorot_uniform(), 'BatchNormalizationV1': BatchNormalization()}): # , 'BatchNormalizationV1': BatchNormalization()
    model = tf.keras.models.load_model(final_model_path + final_model_name)
print(model.summary())
print('model is originally trainable: ' + str(model.trainable))
print('number of layers in the model: ' + str(len(model.layers)))

# set all layers except last two dense ones to be fixed
for layer_idx, layer in enumerate(model.layers):
    if layer_idx > len(model.layers) - 3:
        print(str(layer_idx) + ': ' + str(layer) + ', keeping trainable = ' + str(layer.trainable))
    else:
        layer.trainable = False
        print(str(layer_idx) + ': ' + str(layer) + ', setting trainable to ' + str(layer.trainable))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
1671160358.857963 (InputLaye (None, 17, 4, 1)          0         
_________________________________________________________________
1671160358.858701 (Conv2D)   (None, 17, 4, 32)         1600      
_________________________________________________________________
1671160358.871562 (Flatten)  (None, 2176)              0         
_________________________________________________________________
1671160358.8757818 (Dense)   (None, 256)               557312    
_________________________________________________________________
1671160359.024989 (Dense)    (None, 2)                 514       
Total params: 559,426
Trainable params: 559,426
Non-trainable params: 0
_________________________________________________________________
None
model is originally trainable: True
number of layers in the model: 5
0: <tensorflow.python.keras.engine.input_layer.InputLayer obje

In [7]:
# Transform the test set RBS data to fine-tune this model
data_folder = './clean_data/clean/'
data_file = 'hollerer_rbs_test.csv'

# Give inputs for data generation
input_col = 'seq'
target_col = 'out'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'binary_classification'
model_type = 'deepswarm'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# transform output (target) into bins for classification
transformed_output, transform_obj = transform_classification_target(df_data_output, multiclass = 'multiclass' in task)

# now, we have completed the pre-processing needed to feed our data into deepswarm
# deepswarm input: numerical_data_input
# deepswarm output: transformed_output
X = numerical_data_input
y = to_categorical(transformed_output)

Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [8]:
finetune_model_name = 'fine_tune_deepswarm_deploy_model.h5'
    
print('Fitting final model now...')
num_epochs = 30 # can choose how many epochs you want
deploy_model = fit_final_deepswarm_model(model, task, num_epochs,  X, y)
        
# Save the final deploy trained model
deploy_model.save(final_model_path + finetune_model_name)
print_summary(deploy_model, final_model_path + 'fine_tune_model_topology.txt')
print(deploy_model.summary())

Fitting final model now...
Train on 24888 samples, validate on 2766 samples
Epoch 1/30
24888/24888 [==============================] - 8s 328us/sample - loss: 0.2824 - acc: 0.9122 - binary_crossentropy: 0.2824 - val_loss: 0.2153 - val_acc: 0.9168 - val_binary_crossentropy: 0.2153
Epoch 2/30
24888/24888 [==============================] - 7s 267us/sample - loss: 0.1396 - acc: 0.9539 - binary_crossentropy: 0.1396 - val_loss: 0.2221 - val_acc: 0.9187 - val_binary_crossentropy: 0.2221
Epoch 3/30
24888/24888 [==============================] - 6s 241us/sample - loss: 0.0830 - acc: 0.9767 - binary_crossentropy: 0.0830 - val_loss: 0.2398 - val_acc: 0.9176 - val_binary_crossentropy: 0.2398
Epoch 4/30
24888/24888 [==============================] - 6s 233us/sample - loss: 0.0476 - acc: 0.9888 - binary_crossentropy: 0.0476 - val_loss: 0.2563 - val_acc: 0.9197 - val_binary_crossentropy: 0.2563
_________________________________________________________________
Layer (type)                 Output Shape 

# Example 2: Transfer Learning on an AutoKeras Model 

In [9]:
data_folder = './clean_data/clean/'
data_file = 'hollerer_rbs_test.csv'

# Give inputs for data generation
input_col = 'seq'
target_col = 'out'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'nucleic_acid'
task = 'binary_classification'
model_type = 'autokeras'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# Format data inputs appropriately for autoML platform
transformed_output, transform_obj = transform_classification_target(df_data_output, multiclass = 'multiclass' in task)

# now, we have completed the pre-processing needed to feed our data into autokeras
# autokeras input: oh_data_input
# autokeras output: transformed_output
X = oh_data_input
y = transformed_output # don't convert to categorical for autokeras


Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [10]:
final_model_path = './exemplars/rbs/models/autokeras/binary_classification/'
final_model_name = 'optimized_autokeras_pipeline_classification.h5'

In [11]:
train_size = 0.85
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, np.array(y).astype(float),train_size=train_size, test_size = 1-train_size)

clf = autokeras.utils.pickle_from_file(final_model_path+final_model_name)

evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
print('Evaluation after no retraining: ', evaluation)

# retrain = False indicates that the weights should be reused and then retrained
# retrain = True indicates that the weights should be reinitialized from scratch
# this may seem unintuitive but in the documentation, retrain is a boolean indicating whether or not to reinitialize the weights of the model
clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=False)
evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
print('Evaluation after some retraining: ', evaluation)

# can save and reload at will
autokeras.utils.pickle_to_file(clf, final_model_path + 'fine_tune_autokeras_pipeline_classification.h5')
test = autokeras.utils.pickle_from_file(final_model_path+'fine_tune_autokeras_pipeline_classification.h5')

# showing retrain = True wipes the old weights and ends up with a worse model
clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=True)
evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))
print('Evaluation after training weights from scratch: ', evaluation)

Evaluation after no retraining:  0.91588334538443
Evaluation after some retraining:  0.9149192576524464
Evaluation after training weights from scratch:  0.8879248011569053


# Part 3: Transfer Learning on TPOT Model

In [13]:
# read in data file
data_folder = './clean_data/clean/'
data_file = 'classification_test_peptides.csv'

# give inputs for data generation
input_col_name = 'seq'
target_col = 'target'
pad_seqs = 'max'
augment_data = 'none'
sequence_type = 'protein'
task = 'regression'
model_type = 'tpot'

# allows user to interpret model with data not in the original training set
# so apply typical cleaning pipeline
df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)
    
# format data inputs appropriately for autoML platform    
numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)

# Format data inputs appropriately for autoML platform
transformed_output, transform_obj = transform_regression_target(df_data_output)

X = numerical_data_input
y = transformed_output # don't convert to categorical for tpot
training_features, training_target = reformat_data_traintest(X, y)
train_size = 0.85
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(training_features, training_target, train_size=train_size, test_size = 1-train_size)

Example of bad letter X: JJHKPQAKSYXPYRILDYJJ
Example of bad letter J: JJHKPQAKSYLAYRILDYJJ
Replacing J with substitution : L, I
Setting all substitutions to 1 in one-hot encoded representation...
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.


In [14]:
# give inputs for paths
final_model_path = './exemplars/peptides/outputs/tpot/regression/'
final_model_name = 'final_model_tpot_regression.pkl'
output_folder = final_model_path

with open(final_model_path+final_model_name, 'rb') as file:  
    model = pickle.load(file)

In [15]:
# partial_fit transfer learning is only possible for models that support it - most do not
# see reference list of those models here: https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning
try:
    model.partial_fit(X_train_new,y_train_new)
except:
    print("No partial_fit could be applied. Trying warm_start instead.")
    print("")
try:
    # Can check out the original model parameters - should see warm_start = False
    # print(model.get_params())
    preds = model.predict(X_test_new)
    
    print('Original model on new test data: ', sp.pearsonr(y_test_new, preds))
    
    print('Keys that must be manually changed in the model to allow fine-tuning on new data: ')
    for key in list(model.get_params().keys()):
        if 'warm_start' in key or 'n_estimator' in key:
            print('\t' + key)
        model.set_params(warm_start = True)
        model.set_params(n_estimators = 1 + model.get_params()['n_estimators'])

        #model.set_params(stackingestimator__estimator__warm_start = True)
        #model.set_params(extratreesregressor__warm_start = True)
        #model.set_params(stackingestimator__estimator__n_estimators = 1 + model.get_params()['stackingestimator__estimator__n_estimators'])
        #model.set_params(extratreesregressor__n_estimators = 1 + model.get_params()['extratreesregressor__n_estimators'])
    
    # Can check out the new model parameters - should see warm_start = True and n_estimators higher 
    # n_estimators must be increased because you need to allow new estimators to be created)
    # see reference here for more information: https://stackoverflow.com/questions/42757892/how-to-use-warm-start
    # print(model.get_params())
    model.fit(X_train_new,y_train_new)
    preds = model.predict(X_test_new)    
    print('Fine-tuned model on new test data: ', sp.pearsonr(y_test_new, preds))
except Exception as e:
    print(e)
    print("No warm_start could be applied. Model is not compatible with transfer learning.")


No partial_fit could be applied. Trying warm_start instead.

Original model on new test data:  (0.6139186709056942, 1.2461480191425559e-08)
Keys that must be manually changed in the model to allow fine-tuning on new data: 
	n_estimators
	warm_start
Fine-tuned model on new test data:  (0.6314972307097066, 3.5335892412258266e-09)


# Part 4: Transfer Learning on AutoKeras Toehold Regression Model + Testing

In [3]:
task = 'regression'
pad_seqs = 'none'
augment_data = 'none'
sequence_type = 'nucleic_acid'
model_type = 'autokeras'
final_model_path = './exemplars/toeholds/models/autokeras/regression/'

## With retrained Green et al. models, predict on test sets (Pardee et al.)

In [4]:
pearsons = []
spearmans = []
for i in range(25): # do any # of trials
    print("--------------  ROUND " + str(i) + " --------------")
    # Read in Green et al. data file
    data_folder = './clean_data/clean/'
    data_file = 'green_sequences_toehold_test_set.csv'

    # Give inputs for data generation
    input_col = 'seq'
    target_col = 'target'
    final_model_name = 'optimized_autokeras_pipeline_regression.h5'

    # allows user to interpret model with data not in the original training set
    # so apply typical cleaning pipeline
    df_data_input, df_data_output, _ = read_in_data_file(data_folder + data_file, input_col, target_col)

    # Format data inputs appropriately for autoML platform
    numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)
    transformed_output, transform_obj = transform_regression_target(df_data_output)

    # now, we have completed the pre-processing needed to feed our data into autokeras
    # autokeras input: oh_data_input
    # autokeras output: transformed_output
    X = oh_data_input
    y = transformed_output # don't convert to categorical for autokeras

    train_size = 0.9
    X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, np.array(y).astype(float),train_size=train_size, test_size = 1-train_size)

    clf = autokeras.utils.pickle_from_file(final_model_path+final_model_name)
    evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))

    # retrain = False indicates that the weights should be reused and then retrained
    # retrain = True indicates that the weights should be reinitialized from scratch
    clf.fit(np.array(X_train_new),np.array(y_train_new), retrain=False)
    evaluation = clf.evaluate(np.array(X_test_new), np.array(y_test_new))

    # can save and reload at will
    autokeras.utils.pickle_to_file(clf, final_model_path + 'fine_tune_autokeras_pipeline_classification.h5')

    # Read in data file
    data_dir = './clean_data/clean/'
    file_name = 'clean_pardee_sequences_toehold_test_set.csv'

    # Give inputs for data generation
    input_col = 'seq'
    target_col = 'rank'
    df_data_input, df_data_output, _ = read_in_data_file(data_dir + file_name, input_col, target_col)
        
    # Give inputs for paths
    final_model_name = 'fine_tune_autokeras_pipeline_classification.h5'

    # Format data inputs appropriately for autoML platform 
    numerical_data_input, oh_data_input, df_data_output, scrambled_numerical_data_input, scrambled_oh_data_input, alph = convert_generic_input(df_data_input, df_data_output, pad_seqs, augment_data, sequence_type, model_type = model_type)
    preds = AutoMLBackend.generic_predict(oh_data_input, numerical_data_input, model_type, final_model_path, final_model_name)

    # We are interested in class 1
    preddf = pd.DataFrame(preds)
    y_pred = preddf.iloc[:,0]
    y_true = list(df_data_output.iloc[:,0]) # use output that was scrambled in same order
    
    # do metrics
    slope, intercept, r_val, p_val, std_error = sp.linregress(y_true, y_pred)
    pearsonr = sp.pearsonr(y_true, y_pred)
    spearman = sp.spearmanr(y_true, y_pred)
    pearsons.append(pearsonr[0])
    spearmans.append(spearman[0])

--------------  ROUND 0 --------------
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
--------------  ROUND 1 --------------
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
--------------  ROUND 2 --------------
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, a

Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
--------------  ROUND 20 --------------
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
--------------  ROUND 21 --------------
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: No data augmentation requested
Confirmed: Scrambled control generated.
Confirmed: All sequence characters are in alphabet
Confirmed: No need to pad or truncate, all sequences same length
Confirmed: N

In [52]:
print(np.mean(pearsons))
print(np.std(pearsons))
print(np.mean(spearmans))
print(np.std(spearmans))

0.31852604570850174
0.04956622474192118
0.25735652173913043
0.06663728754036465
